In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

/data/jiacheng/miniconda3/envs/common/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [3]:
INPUT_DIR = "/home/ljc/data/graphrag/ragtest6/output/20240903-120120/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [4]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 906


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
0,0,UNITED STATES OF AMERICA,GEO,"The United States of America, commonly known a...",cc0dbbd456ec94f96f25793657991734,8,15,0,b45241d70f0e43fca764df95b2b81f77,15.0,"[-0.11693376302719116, 0.04598860442638397, 0....",None,b45241d70f0e43fca764df95b2b81f77,27.067825,26.484116
1,0,"WASHINGTON, D.C.",GEO,"Washington, D.C. is the capital city and feder...","aa86232b2bed138127746cc93ab4c0bc,cc0dbbd456ec9...",8,19,1,4119fd06010c494caa07f439b333f4c5,19.0,"[-0.10667740553617477, -0.00014335523883346468...",None,4119fd06010c494caa07f439b333f4c5,9.129201,17.041071
2,0,CANADA,GEO,Canada is a country that borders the United St...,cc0dbbd456ec94f96f25793657991734,8,1,2,d3835bf3dda84ead99deadbeac5d0d7d,1.0,"[-0.08296206593513489, 0.029930362477898598, 0...",None,d3835bf3dda84ead99deadbeac5d0d7d,28.114258,27.530676
3,0,MEXICO,GEO,Mexico is a country that borders the United St...,cc0dbbd456ec94f96f25793657991734,8,1,3,077d2820ae1845bcbb1803379a3d1eae,1.0,"[-0.07760462164878845, 0.036451857537031174, 0...",None,077d2820ae1845bcbb1803379a3d1eae,27.245827,26.662113
4,0,ALASKA,GEO,Alaska is a state of the United States located...,cc0dbbd456ec94f96f25793657991734,8,1,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,1.0,"[-0.08033616840839386, 0.02773427590727806, 0....",None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,28.256557,27.673010


#### Read relationships

In [5]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 297


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,UNITED STATES OF AMERICA,"WASHINGTON, D.C.",9.0,"Washington, D.C. is the federal capital distri...",[cc0dbbd456ec94f96f25793657991734],42be4e140061482ea509dd3e26189480,0,15,19,34
1,UNITED STATES OF AMERICA,CANADA,7.0,Canada borders the United States to the north,[cc0dbbd456ec94f96f25793657991734],4da4ef951ff340f1a3dd679de4be3341,1,15,1,16
2,UNITED STATES OF AMERICA,MEXICO,7.0,Mexico borders the United States to the south,[cc0dbbd456ec94f96f25793657991734],2f05fcce857e4a499ca4e89a3cefbcb3,2,15,1,16
3,UNITED STATES OF AMERICA,ALASKA,8.0,Alaska is a state of the United States located...,[cc0dbbd456ec94f96f25793657991734],b3aeb7ae009a4f52ae3ae4586e32fe11,3,15,1,16
4,UNITED STATES OF AMERICA,HAWAII,8.0,"Hawaii is a state of the United States, an arc...",[cc0dbbd456ec94f96f25793657991734],089b9b9841714b8da043777e2cda3767,4,15,1,16


In [6]:
# NOTE: covariates are turned off by default, because they generally need prompt tuning to be valuable
# Please see the GRAPHRAG_CLAIM_* settings
# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

# claims = read_indexer_covariates(covariate_df)

# print(f"Claim records: {len(claims)}")
# covariates = {"claims": claims}

FileNotFoundError: [Errno 2] No such file or directory: '/home/ljc/data/graphrag/ragtest6/output/20240903-120120/artifacts/create_final_covariates.parquet'

#### Read community reports

In [11]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
print(reports)

report_df.head()

Report records: 30
[CommunityReport(id='28', short_id='28', title='Wall Street and Financial District', community_id='28', summary="The community is centered around Wall Street, located in the Financial District of Manhattan, New York City. Wall Street is a globally recognized financial hub, housing major stock exchanges such as the New York Stock Exchange and Nasdaq, and significantly contributes to New York City's status as an economic powerhouse.", full_content="# Wall Street and Financial District\n\nThe community is centered around Wall Street, located in the Financial District of Manhattan, New York City. Wall Street is a globally recognized financial hub, housing major stock exchanges such as the New York Stock Exchange and Nasdaq, and significantly contributes to New York City's status as an economic powerhouse.\n\n## Wall Street as a global financial center\n\nWall Street is renowned as the world's premier financial and fintech center, hosting major stock exchanges like the Ne

,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,28,# Wall Street and Financial District\n\nThe co...,2,9.5,Wall Street and Financial District,The impact severity rating is high due to Wall...,"The community is centered around Wall Street, ...",[{'explanation': 'Wall Street is renowned as t...,"{\n ""title"": ""Wall Street and Financial Dis...",95cdb85f-726f-4e32-9633-e96e91d1e707
1,29,# New York City and Its Global Influence\n\nNe...,2,9.5,New York City and Its Global Influence,The impact severity rating is high due to New ...,"New York City, the most populous city in the U...",[{'explanation': 'New York City is a major glo...,"{\n ""title"": ""New York City and Its Global ...",7a3fe03a-1d92-4d0f-9ff2-8c8635b5d9ce
2,11,# France and its Global Influence\n\nThe commu...,1,9.0,France and its Global Influence,The impact severity rating is high due to Fran...,"The community centers around France, a key pla...",[{'explanation': 'France is a founding and lea...,"{\n ""title"": ""France and its Global Influen...",724b863f-2114-4598-aec5-ef72919290cd
3,12,# European Union and Its Foundational Entities...,1,9.5,European Union and Its Foundational Entities,The impact severity rating is high due to the ...,The European Union (EU) is a significant supra...,[{'explanation': 'The European Union (EU) is a...,"{\n ""title"": ""European Union and Its Founda...",bef28e42-3018-4d28-aaa0-1568b061ba57
4,13,"# Seattle: A Hub of Culture, Technology, and H...",1,8.5,"Seattle: A Hub of Culture, Technology, and His...",The impact severity rating is high due to Seat...,Seattle is a major city on the West Coast of t...,[{'explanation': 'Seattle is home to major tec...,"{\n ""title"": ""Seattle: A Hub of Culture, Te...",0ef7aa47-2d4f-4454-8095-c0331cb2fe6e


#### Read text units

In [14]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 16


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,cc0dbbd456ec94f96f25793657991734,"The United States of America (USA or U.S.A.), ...",669,[0134252ef584e1c209f4b4937c1d7a4f],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[42be4e140061482ea509dd3e26189480, 4da4ef951ff..."
1,b324dfcc8a3699591c60646ccaf6e4b0,Nice (/niːs/ NEESS; French pronunciation: [nis...,1035,[10dbe8991bb9a7ab1dda37e9ccbedd78],"[e657b5121ff8456b9a610cfaead8e0cb, bf4e255cdac...","[bdddcb17ba6c408599dd395ce64f960a, bc70fee2061..."
2,c796265ad38bbdc79dc17b2d0aba7e9e,"New York, often called New York City[b] or NYC...",746,[147a4bae12320a91e5843421457c4c0e],"[bc0e3f075a4c4ebbb7c7b152b65a5625, 958beecdb5b...","[4d183e7007624fcd98af96b9d752c16d, 57b8930790c..."
3,e201254a65d111262b3361be30dbaa31,The European Union (EU) is a supranational pol...,621,[45118b491c6d36b4a415322d2c444105],"[c03ab3ce8cb74ad2a03b94723bfab3c7, ed6d2eee9d7...","[453ecf5476f64f4a8d5020b95baf1314, 6a1d83c9ce2..."
4,7f6347c263ce28650c650f3f050a4f70,Paris (French pronunciation: [paʁi] ⓘ) is the ...,825,[474b35b12bfb44f1235cd856dcae3b43],"[b7702b90c7f24190b864e8c6e64612a5, 6b02373137f...","[92646910ee624bd7909fac2b5c0232e3, ccbbbcc055c..."


In [7]:
api_key = 
llm_model = 'gpt-4o-2024-08-06'
embedding_model = 'text-embedding-3-small'

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

### Create local search context builder

In [16]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    # covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [17]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [18]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [20]:
result = await search_engine.asearch("Tell me about Paris")
print(result.response)

# Overview of Paris

Paris, the capital and largest city of France, is a global hub renowned for its significant roles in finance, diplomacy, commerce, culture, fashion, and gastronomy. Often referred to as the "City of Light," Paris has been a major center for arts and sciences since the 17th century. The city is the heart of the Île-de-France region, which is the most populous region in France and a major economic powerhouse within the European Union [Data: Entities (92); Relationships (119, 135)].

## Cultural Significance

Paris is home to some of the world's most famous cultural institutions. The Louvre, the most-visited art museum globally, and the Musée d'Orsay, known for its French Impressionist art collection, are key attractions that contribute to Paris's reputation as a cultural capital. Other notable museums include the Pompidou Centre, Musée Picasso, and Musée Marmottan Monet, which are celebrated for their collections of modern and contemporary art [Data: Entities (98, 99

In [ ]:
question = "Tell me about Dr. Jordan Hayes"
result = await search_engine.asearch(question)
print(result.response)

#### Inspecting the context data used to generate the response

In [ ]:
result.context_data["entities"].head()

In [ ]:
result.context_data["relationships"].head()

In [ ]:
result.context_data["reports"].head()

In [ ]:
result.context_data["sources"].head()

In [ ]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [21]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [22]:
question_history = [
    "Tell me about Paris",
    "What happens in Paris 2024?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)

['- What major international events are scheduled to take place in Paris in 2024?', '- How is Paris preparing for the 2024 Summer Olympics?', "- What impact will the 2024 Summer Olympics have on Paris's economy and infrastructure?", '- Which venues in Paris will host events during the 2024 Summer Olympics?', '- How will the 2024 Summer Olympics affect tourism in Paris?']
